Experimenting with Bayesian optimisation to produce adversarial examples.


References:
- General guide to bayesian optimization:
    https://distill.pub/2020/bayesian-optimization/
- Paper on using bayesian optimization in hard label settings:
https://arxiv.org/pdf/2007.07210.pdf
- Another paper with a different objective function that could be used:
https://arxiv.org/abs/1807.04457

Things to consider:
- Bayesian optimisation performs best on problems with a low number of dimensions (eg under 20) - our problem has on the scale of ~100 dimensions, which is still significantly less than image recognition problems. Other techniques (dimensional upsampling) can be used to use bayesian optimization on high dimension problems - may or may not be necessary here
- Search space needs to be defined - initially take mins/max of training data for each field? This will later be refined (eg changing derived fields along with postcode)
- Initial samples from which adversarial examples are to be generated - just take failed companies from the test data?
- skopt includes callbacks - monitor process of the optimisation/terminate early when we find an adversarial example.

Scikit-optimize: library for optimization, including functions for Bayesian optimization.

In [20]:
import skopt
import os
import pickle
import pandas as pd
import numpy as np

Step 1: defining search space.
Initially do so by taking min/max for each field in test data.

In [21]:
#Load in test data
data_dir = os.getcwd()
with open(os.path.join(data_dir, "Processing", r"test_preproc.p"), 'rb') as data_file:
    test_data = pickle.load(data_file)
X_test, y_test = test_data

In [22]:
#Of shape (no. samples, no. features)
X_test.shape

(120000, 115)

In [23]:

X_test.iloc[0].shape

(115,)

In [24]:
min_max = pd.DataFrame(index=["min", "max"], columns=X_test.columns)

for column in X_test.columns:
    min_max[column]["max"] = X_test[column].max(axis=0)
    min_max[column]["min"] = X_test[column].min(axis=0)

    
pd.set_option("display.max_rows", 345)
min_max.transpose()

,min,max
AccountsAccountCategory,0,15
AccountsAccountRefDay,1,31
AccountsAccountRefMonth,1,12
CompanyCategory,0,1
CompanyNameCountNum,0,26
CompanyNameCountX,0,3
CompanyNameLen,6,114
CompanyNameWordLen,1,16
Field1014,-2.25523,5.02972
Field1129,-2.19484,2.73187


In [25]:
#load in model to attack
model_name = "rf_0.929"
with open(os.path.join(data_dir, "fitted_models", model_name), 'rb') as data_file:
    model = pickle.load(data_file)

In [26]:
# Need to get search space from min_max array
search_space = list(map(tuple, min_max.transpose().to_records(index=False)))

In [117]:
# define objective function
# to think about: minimising delta? check if delta<epsilon, if not return -1?
# X is a list of parameters
def obj_func(X):
    X_array = np.array(X).reshape(1,-1)
    y1 = model.predict(X_array)
    if(y1 == 0):
        print("not right")
        return 0
    else:
        print("gottem!!")
        print("y1: %s" % y1)
        return -1

In [118]:
class FinishedCallBack(skopt.callbacks.EarlyStopper):
    def __call__(self, result):
      """
      Messy code
      Parameters
      ----------
      result : `OptimizeResult`, scipy object
          The optimization as a OptimizeResult object.
      """
      return self._criterion(result)
    def _criterion(self, result):
        print(result.fun)
        if(result.fun == -1):
            print("I should stop now")
        return (result.fun == -1)

In [119]:
# get initial point (company data we want to adjust)
# for now, just use the first company in the test data that is predicted to fail
fail = False
i = 0
while(not(fail)):
    if(y_test[i] == 0):
        if(model.predict(X_test.iloc[i].values.reshape(1, -1)) == 0):
            example = X_test.iloc[i].values.reshape(1,-1)
            fail = True
    if (not(fail)): i += 1

print(model.predict(example))
print(y_test[i])
example=example.flatten().tolist()
len(example)
example
obj_func(example)

[0]
0
not right


0

In [144]:
# sometimes gets error here - keep rerunning and it will work after a while.
cb = FinishedCallBack()
res = skopt.gp_minimize(obj_func, search_space, x0 = example, y0 = 0, callback=cb)
minimum = res.x

0
not right
0
gottem!!
y1: [1]
-1
I should stop now


In [145]:
minimum


[11,
 16.508499989873982,
 10.75753262078372,
 0,
 15,
 3,
 74,
 13,
 2.1471106226426304,
 2.6181618028709748,
 1.5906235195082243,
 1.435488294109411,
 -0.8791334196253706,
 -1.0238313025169639,
 -1.1672291062782083,
 -3.1027982722866883,
 -1.7095690597033582,
 10,
 0.46583682079482536,
 0.03280497776384883,
 1.4736122531714124,
 1.445573886604462,
 0.5505867868364831,
 6.238614282617944,
 2.3946182495550676,
 0.3851268483214203,
 0.31267688279847405,
 -1.9954449418125355,
 3.1484862639171936,
 -2.3172802471577287,
 590,
 -0.25646125395310126,
 295255,
 2.4342812978880013,
 1.7158694411531044,
 0.9425665933041718,
 -0.15197183679386583,
 0.5756215150421984,
 0.453796668855458,
 1.1178225627217593,
 1.2186121958048597,
 1.9398604284382013,
 0.952520208184013,
 1,
 -13.496363985471662,
 1164,
 448,
 2,
 1025,
 2,
 26.117623397589988,
 5.896227131928319,
 52.75393786695051,
 2,
 28,
 1997.6439636218065,
 1988.727782511576,
 2017.7062255425349,
 2019.6882021599142,
 2013.2273809761093,
 1

In [148]:
# Sanity check - make sure the model actually outputs 1 for the example found
model.predict(np.array(minimum).reshape(1,-1))

array([1])

In [149]:
# Check the difference - pretty big
np.linalg.norm(np.array(minimum)-np.array(example))

15348.44668582168

Next problem - minimising the difference between starting sample and adversarial example.
How to do that - is there something we can use within bayesian optimisation function? Or can we do another layer of bayesian optimisation to minimise the distance?